In [1]:
import sys
import os
sys.path.append("../../src/")
sys.path.append("../../utilities/")
import dataset_class
from dataset_class import dataset
import data_utils
import evaluation_utils
import numpy as np
import pandas as pd
import seaborn as sns

In [125]:
from importlib import reload
evaluation_utils = reload(evaluation_utils)
data_utils = reload(data_utils)

In [3]:
signals_dir = '../../data/GM12878/hg19/genomic-assays/bin_10000/'
TAD_path = '../../data/GM12878/hg19/annotations/GSE63525_GM12878_primary+replicate_Arrowhead_domainlist.txt'
CTCF_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/CTCF/lifted_data'
RNAPII_bedpe_path = '../../data/GM12878/hg19/ChIA-PET/RNAPII/lifted_data'
gene_expression_path = '../../data/GM12878/hg19/gene_expression/GM12878_genes_RPKM.txt'
RT_path = '../../data/GM12878/hg19/RT/six_phase/six_phases.txt'
TAD_agreements = {}
CTCF_ChIA_PETs = {}
RNAPII_ChIA_PETs = {}
resolution = 10000

In [6]:
annotation_path = '../../data/GM12878/hg19/res10000_datasets/annotations/gmm_mp_0_annotation.txt'

In [96]:
evaluation_utils.get_coverage(annotation_path)

{'0': 0.5730382963276666,
 '3': 0.07854755278601246,
 '2': 0.12260647952084215,
 '4': 0.07565490955290702,
 '1': 0.15015276181257184}

In [ ]:
TADs = pd.read_csv(TAD_path, sep = "\t")

In [114]:
TAD_freqs = []
for ind,row in TADs.iterrows():
    chr_name = 'chr{}'.format(row['chr_name'])
    if not evaluation_utils.is_valid_chr(chr_name):
        continue
    start = int(row['start'])
    end = int(row['end'])
    TAD_freqs.append(evaluation_utils.annot_freq_for_region_by_arm(chr_name, start, end, annotation_dict, resolution))

In [120]:
labels = [str(i) for i in np.arange(5)]
total_TAD_freqs = {}
for label in labels:
    total_TAD_freqs[label] = 0

In [121]:
for freq in TAD_freqs:
    for key in freq:
        if key in labels:
            total_TAD_freqs[key] = total_TAD_freqs[key] + freq[key]

In [126]:
data_utils.normalize_dict(total_TAD_freqs)

{'0': 0.3868355562050573,
 '1': 0.20336375006984533,
 '2': 0.15933326742801637,
 '3': 0.12595795527321338,
 '4': 0.12450947102386777}

In [100]:
TADs = TADs.iloc[:,0:3]
TADs.columns = ['chr_name', 'start', 'end']

In [103]:
TADs.sort_values(['chr_name', 'start'], inplace=True)

In [108]:
TADs[TADs['chr_name']=='1']['start'].values[1:] > TADs[TADs['chr_name']=='1']['end'].values[:-1]

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True, False, False,
        True,  True,  True,  True, False,  True,  True,  True, False,
       False,  True, False, False, False,  True,  True,  True, False,
       False, False,  True,  True, False,  True,  True, False, False,
        True,  True, False, False, False, False,  True,  True,  True,
        True,  True, False, False,  True,  True, False,  True,  True,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True, False,  True,
        True, False,  True,  True,  True, False, False,  True, False,
        True,  True,  True, False,  True, False, False,  True, False,
       False, False,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True, False, False,
       False, False, False,  True, False,  True,  True, False, False,
        True, False,

In [7]:
annotation_dict = evaluation_utils.make_annotation_dict_chr_arm(annotation_path,resolution)

In [77]:
f_most,f_ds, valid_f_ds, num_TADs = evaluation_utils.TAD_agreement(annotation_dict, TAD_path, resolution, 0)

In [78]:
print(len(f_ds), np.mean(f_ds), num_TADs)

9048 0.5570625942417263 9048


In [79]:
p_most,p_ds, valid_p_ds, num_TADs = evaluation_utils.TAD_agreement(annotation_dict, TAD_path, resolution, 0.3)

In [80]:
print(len(p_ds), np.mean(p_ds))

8230 0.6838934351155406


In [87]:
len([f_ds[i] for i in valid_p_ds])

8230

In [89]:
#num_TADs = pd.read_csv(TAD_path, sep = "\t").shape[0]
p_ds_df = pd.DataFrame({'TAD_num': np.arange(num_TADs)})

In [90]:
p_ds_df['p_f_d'] = None
p_ds_df['p_most'] = None
p_ds_df['f_d'] = f_ds
p_ds_df['f_most'] = f_most
p_ds_df['p_f_d'][valid_p_ds] = p_ds
p_ds_df['p_most'][valid_p_ds] = p_most
p_ds_df.dropna(axis=0, inplace=True)

/home/nedooshki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/nedooshki/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
